In [2]:
import os
from datetime import datetime

import statsapi as mlb
from datetime import datetime
import time

import sql_alchemy_mapping

# connect to the database with custom MyDatabase class
# configured for a sqlite DB stored in current working directory 
db = sql_alchemy_mapping.db

# instantiate sql alchemy session and base
session = sql_alchemy_mapping.session
Base = sql_alchemy_mapping.Base

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb_sqlite/mlb.db)


In [3]:
Game = sql_alchemy_mapping.Game
Person = sql_alchemy_mapping.Person
Pitch = sql_alchemy_mapping.Pitch
Play = sql_alchemy_mapping.Play
Team = sql_alchemy_mapping.Team
GameTeamLink = sql_alchemy_mapping.GameTeamLink

In [4]:
from os import walk
import csv
import re
def read_gamePks():
    gamePks_path = os.getcwd()+"/gamePks"
    f = []
    for (dirpath, dirnames, filenames) in walk(gamePks_path):
        f.extend(filenames)
        break
    pk_paths = [gamePks_path + '/' + x for x in f if x[0]!= '.']
    
    gamePks = {}
    for path in pk_paths:
        season = re.findall('/gamePks/([^.csv]+)',path)
        with open(path, 'r') as f:
            reader = csv.reader(f)
            seasonPks = list(reader)
        gamePks[season[0]] = [item for sublist in seasonPks for item in sublist]
    return gamePks

import numpy as np
import random 
gamePk_dict=read_gamePks()
games_2019 = gamePk_dict['2019']

games = random.sample(games_2019,len(games_2019)//30)

In [5]:
count = 1 
for game in games:
    print(f'starting {count} out of {len(games)}')
    Game(game,session,commit=True,verbose=True)
    count+=1

starting 1 out of 300
adding game record
adding play records
commit...
starting 2 out of 300
adding game record
adding play records
commit...
starting 3 out of 300
adding game record
adding play records
commit...
starting 4 out of 300
adding game record
adding play records
commit...
starting 5 out of 300
adding game record
adding play records
commit...
starting 6 out of 300
adding game record
adding play records
commit...
starting 7 out of 300
adding game record
adding play records
commit...
starting 8 out of 300
adding game record
adding play records
commit...
starting 9 out of 300
adding game record
adding play records
commit...
starting 10 out of 300
adding game record
adding play records
commit...
starting 11 out of 300
adding game record
adding play records
commit...
starting 12 out of 300
adding game record
adding play records
commit...
starting 13 out of 300
adding game record
adding play records
commit...
starting 14 out of 300
adding game record
adding play records
commit...
s

adding game record
adding play records
commit...
starting 116 out of 300
adding game record
adding play records
commit...
starting 117 out of 300
adding game record
adding play records
commit...
starting 118 out of 300
adding game record
adding play records
commit...
starting 119 out of 300
adding game record
adding play records
commit...
starting 120 out of 300
adding game record
adding play records
commit...
starting 121 out of 300
adding game record
adding play records
commit...
starting 122 out of 300
adding game record
adding play records
commit...
starting 123 out of 300
adding game record
adding play records
commit...
starting 124 out of 300
adding game record
adding play records
commit...
starting 125 out of 300
adding game record
adding play records
commit...
starting 126 out of 300
adding game record
adding play records
commit...
starting 127 out of 300
adding game record
adding play records
commit...
starting 128 out of 300
adding game record
adding play records
commit...
st

adding game record
adding play records
commit...
starting 229 out of 300
adding game record
adding play records
commit...
starting 230 out of 300
adding game record
adding play records
commit...
starting 231 out of 300
adding game record
adding play records
commit...
starting 232 out of 300
adding game record
adding play records
commit...
starting 233 out of 300
adding game record
adding play records
commit...
starting 234 out of 300
adding game record
adding play records
commit...
starting 235 out of 300
adding game record
adding play records
commit...
starting 236 out of 300
adding game record
adding play records
commit...
starting 237 out of 300
adding game record
adding play records
commit...
starting 238 out of 300
adding game record
adding play records
commit...
starting 239 out of 300
adding game record
adding play records
commit...
starting 240 out of 300
adding game record
adding play records
commit...
starting 241 out of 300
adding game record
adding play records
commit...
st

In [6]:
# get the team ids from the games table
# de-dupe against team ids already in the teams table
# add and commit remaining teams

teams = {item for sublist in 
         session.execute('select homeTeam_id,awayTeam_id from games').fetchall() 
         for item in sublist}

already_added = {item for sublist in 
         session.execute('select id from teams').fetchall() 
         for item in sublist}

teams = list(teams-already_added)

session.add_all(Team(x) for x in teams)
session.commit()

In [7]:
game_ids=session.execute("""select id from games""").fetchall()
game_ids=[item for sublist in game_ids for item in sublist]


def chunk(n,list_to_chunk):
    """
    takes in n, and a list to chunk. returns a list of lists with n length. The last chunk size may or may not 
    be equal to n. 
    """
    return [ list_to_chunk[i:i+n] for i in range(0,len(list_to_chunk),n) ]

chunked_game_ids = chunk(30,game_ids)
count=1

for ids in chunked_game_ids:
    print(f'chunk{count} out of {len(chunked_game_ids)}')
    session.add_all([GameTeamLink(x,'away') for x in ids])
    session.commit()
    count+=1
    

chunk1 out of 10
chunk2 out of 10
chunk3 out of 10
chunk4 out of 10
chunk5 out of 10
chunk6 out of 10
chunk7 out of 10
chunk8 out of 10
chunk9 out of 10
chunk10 out of 10


In [8]:
game = session.query(Game).all()[200]

In [9]:
play = game.plays[0]

In [10]:
[x.startTime for x in play.pitches]

[datetime.datetime(2019, 5, 28, 23, 12, 24),
 datetime.datetime(2019, 5, 28, 23, 12, 25),
 datetime.datetime(2019, 5, 28, 23, 12, 26),
 datetime.datetime(2019, 5, 28, 23, 12, 27),
 datetime.datetime(2019, 5, 28, 23, 12, 28)]

In [11]:
charlie_morton = session.query(Person).filter(Person.id==game.home_probablePitcher).one()

In [12]:
[x.startTime for x in charlie_morton.pitcher_at_bats]

[datetime.datetime(2019, 3, 6, 21, 1, 14),
 datetime.datetime(2019, 3, 6, 21, 4, 50),
 datetime.datetime(2019, 3, 6, 21, 6, 33),
 datetime.datetime(2019, 3, 6, 21, 7, 42),
 datetime.datetime(2019, 3, 6, 21, 9, 46),
 datetime.datetime(2019, 3, 19, 19, 18, 40),
 datetime.datetime(2019, 3, 19, 19, 23, 22),
 datetime.datetime(2019, 3, 19, 19, 24, 56),
 datetime.datetime(2019, 3, 19, 19, 25, 55),
 datetime.datetime(2019, 5, 19, 19, 28, 43),
 datetime.datetime(2019, 5, 19, 19, 34, 10),
 datetime.datetime(2019, 5, 19, 19, 37, 5),
 datetime.datetime(2019, 5, 28, 19, 51, 52),
 datetime.datetime(2019, 5, 28, 23, 12, 49),
 datetime.datetime(2019, 5, 28, 23, 15, 43),
 datetime.datetime(2019, 5, 28, 23, 16, 59),
 datetime.datetime(2019, 5, 28, 23, 38, 5),
 datetime.datetime(2019, 5, 28, 23, 38, 52),
 datetime.datetime(2019, 5, 28, 23, 40, 13),
 datetime.datetime(2019, 5, 28, 23, 43, 12),
 datetime.datetime(2019, 6, 15, 2, 50, 7),
 datetime.datetime(2019, 6, 15, 3, 3, 55),
 datetime.datetime(2019, 6